First, we define the primitives (mathematical functions) we want to use for our tree. To make further implementations easier we limit ourselves to binary operators

In [1]:
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
def div(x, y):
    """
    protected division. Returns 1000000 if dividing by 0
    """
    if y == 0: return 1000000
    return x / y
def pot(x, y): return x ** y
def maxi(x, y): 
    if x >=y: return x
    return y
     
def mini(x,y):
    if x <= y: return x
    return y

PRIMITIVES = [add, sub, mul, div, pot, maxi, mini]
PRIMITIVE_SYMBOLS = ["+", "-", "*","/", "^", "max", "min"]

Then we create a Tree class, which will be the base for our tree. It will have a left and right child, and a value. The value will be a either a primitive or a terminal, and the children will be other trees.

We also define a function to print the tree, and a function to evaluate the tree.

Finally we have a set of functions to perform a crossover.


In [2]:
class Tree:

    # Constructor of class Tree
    def __init__(self,data = None,left = None, right = None):
        self.data = data
        self.left = left
        self.right = right

    def compute_tree(self, terminal_values):
        """
        computes the tree, given a dict of terminal values
        """ 
        if (self.data in PRIMITIVES): 
            return self.data(self.left.compute_tree(terminal_values),
                              self.right.compute_tree(terminal_values))
        elif type(self.data) == int or type(self.data) == float:
            return self.data
        else:
            return terminal_values[self.data]
  
    def __str__(self):
        """
        returns a string representation of the tree,
        using the primitive symbols.
        it overrides the str() and print() operator
        
        """
        if (self.data in PRIMITIVES):
            return ("(" + self.left.__str__() 
                    + str(PRIMITIVE_SYMBOLS[PRIMITIVES.index(self.data)]) 
                    + self.right.__str__() + ")")
        else:
            return str(self.data)
    
    def size(self):
        """
        returns the size of the tree,
        i.e. the number of nodes is has that are not none"""
        if (self.data in PRIMITIVES):
            return 1 + self.left.size() + self.right.size()
        else:
            return 1
    
    def subtree(self, count):
        """
        returns a subtree of the tree, starting at node count"""
        t = Tree()
        count -= 1            
        if count <= 0: # return the current subtree as new tree
            t.data  = self.data
            t.left  = self.left
            t.right = self.right
            return t
        else: # move into subtree recursively                
            if self.left and count <= self.left.size(): 
                return self.left.subtree(count)
                # if the left tree has more nodes than the count 
            elif self.right:
                return self.right.subtree(count - self.left.size())
                # if the right tree has more nodes than the count - size of left tree

    def replace(self, count,subtree):
        """
        the subtree replaces the tree, starting at node count
        """
        count -= 1            
        if count <= 0: 
            self.data  = subtree.data
            self.left  = subtree.left
            self.right = subtree.right

        else: # move into subtree recursively              
            if self.left and count <= self.left.size():
                return self.left.replace(count,subtree)
                # if the left tree has more nodes than the count 
            elif self.right:
                return self.right.replace(count - self.left.size(),subtree)
                # if the right tree has more nodes than the count - size of left tree

    def crossover(self, other, point):
        """
        xo 2 trees at node point, make sure the node point exists in both trees.

        Disclaimer: this is only one way of doing a crossover, and might not be the best way.
        """

        assert point <= self.size() and point <= other.size(),"these points do not exist in both trees"
        subtree1 = self.subtree(point)
        subtree2 = other.subtree(point)
        self.replace(point,subtree2)
        other.replace(point,subtree1)
        return self, other

Next, we need to define the fitness function, which will be the mean squared error of the tree on the sample data. In scheduling this will require te simualtion we which is a lot more time intensive in run time.

In [3]:
def determine_fitness(Tree, data):
    fitness = 0
    for i in data:
        terminal_values = i["values"]
        target = i["target"]
        prediction = Tree.compute_tree(terminal_values)
        fitness += (target - prediction)**2
    return fitness/len(data)

Before we can generate the general loop of GP and train it we need a way to generate the initial population. We will do this by generating random trees.

In [4]:
import random
import copy

def random_tree(max_depth, terminals):
    """
    returns a random tree
    """
    if max_depth == 0:
        return Tree(random.sample(terminals,1)[0])
    else:

        return Tree(random.sample(PRIMITIVES,1)[0],
                    random_tree(max_depth-1, terminals),
                    random_tree(max_depth-1, terminals))

def init_population(pop_size, max_depth, terminals, data):
    """
    initializes a population of random trees
    """
    population = []
    for i in range(pop_size):
        tmp_tree = random_tree(random.sample(range(2,max_depth+1), 1)[0], terminals)
        population.append([copy.deepcopy(tmp_tree), determine_fitness(tmp_tree, data)])
    return population

def tournament_selection(population, tournament_size):
    """
    selects the best tree from a tournament
    """
    tournament = random.sample(population, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=False)
    return copy.deepcopy(tournament[0][0])



Finally, we build the general GP loop. we generate a random polulation of trees, and then evolve them. We will use no mutation for simplicty reasons.

In general it would make sense to put all of this into a GP class. but for the purpose of explaining we split it up into singular functions.

In [5]:
def run_gp(generations, popsize, terminals, data,
           tournament_size=5, max_depth=5, elitism=2):

    pop = init_population(popsize, max_depth, terminals, data)
    pop.sort(key = lambda x: x[1], reverse=False)

    print("Generation: 0 Best fitness: ", pop[0][1])

    for gen in range(1, generations + 1):
        new_pop = []
        for i in range(popsize//2):
            indicator = random.random()
            if indicator < 0.1: # keep some old trees
                new_pop.append(copy.deepcopy(pop[random.sample(range(popsize),1)[0]]))
                new_pop.append(copy.deepcopy(pop[random.sample(range(popsize),1)[0]]))

            elif random.random() < 0.9: # crossover
                parent1 = tournament_selection(pop, tournament_size)
                parent2 = tournament_selection(pop, tournament_size)
                point = random.sample(range(1, min(parent1.size(), parent2.size()) + 1),1)[0]
                child1, child2 = parent1.crossover(parent2, point)
                new_pop.append((copy.deepcopy(child1), determine_fitness(child1, data)))
                new_pop.append((copy.deepcopy(child2), determine_fitness(child2, data)))
                
            else: # mutation by crossover with a random tree
                parent1 = tournament_selection(pop, tournament_size)
                parent2 = random_tree(random.sample(range(2,max_depth+1), 1)[0], terminals)
                point = random.sample(range(1, min(parent1.size(), parent2.size()) + 1),1)[0]
                child1, child2 = parent1.crossover(parent2, point)
                new_pop.append((copy.deepcopy(child1), determine_fitness(child1, data)))
                new_pop.append((copy.deepcopy(child2), determine_fitness(child2, data)))
        
        # add elitism keep the best x trees from the old population
        for i in range(elitism):
            new_pop.append(pop[i])
        new_pop.sort(key=lambda x: x[1], reverse=False)
        pop = new_pop[:popsize] # cut of the two worse trees
        print("Generation: ", gen, "Best fitness: ", pop[0][1])

    print("Best tree: ", pop[0][0])
    print("Fitness: ", pop[0][1])



Lets test our GP in finding the function f(x) = x^2 + x*y + y

first we generate the data

In [6]:
data = []
size = 1000
for i in range(size):
    x = random.randint(0,100)
    y = random.randint(0,100)
    data.append({"values": {"x": x, "y": y},
                 "target": x**2 + x*y + y})


Now we can final test our GP. We limit the possible primitives to the simpler ones. 

In [9]:
PRIMITIVES = [add, sub, mul, div]
PRIMITIVE_SYMBOLS = ["+", "-", "*","/"]
run_gp(30, 100, ["x","y"], data)

Generation: 0 Best fitness:  19716245.3
Generation:  1 Best fitness:  5987987.071
Generation:  2 Best fitness:  3377.658
Generation:  3 Best fitness:  3377.658
Generation:  4 Best fitness:  1683.976
Generation:  5 Best fitness:  1683.976
Generation:  6 Best fitness:  1683.976
Generation:  7 Best fitness:  1683.976
Generation:  8 Best fitness:  1683.976
Generation:  9 Best fitness:  0.0
Generation:  10 Best fitness:  0.0
Generation:  11 Best fitness:  0.0
Generation:  12 Best fitness:  0.0
Generation:  13 Best fitness:  0.0
Generation:  14 Best fitness:  0.0
Generation:  15 Best fitness:  0.0
Generation:  16 Best fitness:  0.0
Generation:  17 Best fitness:  0.0
Generation:  18 Best fitness:  0.0
Generation:  19 Best fitness:  0.0
Generation:  20 Best fitness:  0.0
Generation:  21 Best fitness:  0.0
Generation:  22 Best fitness:  0.0
Generation:  23 Best fitness:  0.0
Generation:  24 Best fitness:  0.0
Generation:  25 Best fitness:  0.0
Generation:  26 Best fitness:  0.0
Generation:  27 

As we can see the GP can get stuck in a close to 0 solution easily. Thats why a lot of literature exists to help it get out of local optima. In our simple example rerunning it helps.